In [2]:
import pandas as pd

train = pd.read_csv(r"C:\Projects\ml_experiments\Datasets\fake_news\train.csv")

In [3]:
train

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [4]:
train['label'].value_counts()

label
1    10413
0    10387
Name: count, dtype: int64

In [5]:
train = train.drop(columns=['id', 'title', 'author'], axis=1)

In [6]:
train = train.dropna(axis=0)

In [7]:
import contractions
import string
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [8]:
train['clean_news'] = train['text'].str.lower()
train['clean_news'] = train['clean_news'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
train['clean_news'] = train['clean_news'].apply(lambda x: " ".join(x.split()))
train['clean_news'] = train['clean_news'].apply(lambda x: contractions.fix(x))
train['clean_news'] = train['clean_news'].apply(lambda x: " ".join([word for word in x.split() if word not in stop]))


In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train['clean_news'])
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

In [11]:
vocab_size

266886

In [12]:
X_seq = tokenizer.texts_to_sequences(train['clean_news'])
X_padded = pad_sequences(X_seq, padding='post')

In [13]:
X_padded = pad_sequences(X_padded, maxlen=500, truncating='post')

In [14]:
X_padded.shape

(20761, 500)

In [15]:
embedding_dim = 100  # 100D GloVe vectors
embedding_index = {}

In [16]:
with open(r"C:\Projects\ml_experiments\Datasets\fake_news\glove.6B.100d.txt", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embedding_index[word] = coefs

In [17]:
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if word in embedding_index:
        embedding_matrix[i] = embedding_index[word]

In [18]:
embedding_matrix.shape

(266886, 100)

In [19]:
vocab_size = 266886  # Total vocabulary size
embedding_dim = 100  # Each word is represented as a 300D vector (common for GloVe)
max_length = 500  # Truncate/pad sequences to 500 tokens (instead of 20874 for efficiency)

In [39]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_length),
    LSTM(128, return_sequences=True),  # LSTM layer with 128 units
    Dropout(0.3),  # Regularization to prevent overfitting
    LSTM(64),  # Another LSTM layer with 64 units
    Dropout(0.3),
    Dense(32, activation='relu'),  # Fully connected layer
    Dense(1, activation='sigmoid')  # Output layer (for binary classification)
])

c:\Users\vishwas.balkundi\miniforge3\envs\env_ml\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [40]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_padded, np.array(train['label']), epochs=5)

Epoch 1/5
588/649 ━━━━━━━━━━━━━━━━━━━━ 46s 769ms/step - accuracy: 0.6381 - loss: 0.6281

testing the model

In [31]:
test = pd.read_csv(r"C:\Projects\ml_experiments\Datasets\fake_news\test.csv")
label_test = pd.read_csv(r"C:\Projects\ml_experiments\Datasets\fake_news\submit.csv")

In [32]:
test['label'] = label_test['label']
test = test.drop(columns=['id', 'title', 'author'], axis=1)
test = test.dropna(axis=0)

In [33]:
test['clean_news'] = test['text'].str.lower()
test['clean_news'] = test['clean_news'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
test['clean_news'] = test['clean_news'].apply(lambda x: " ".join(x.split()))
test['clean_news'] = test['clean_news'].apply(lambda x: contractions.fix(x))
test['clean_news'] = test['clean_news'].apply(lambda x: " ".join([word for word in x.split() if word not in stop]))

In [34]:
X_test_seq = tokenizer.texts_to_sequences(test['clean_news'])
X_test_padded = pad_sequences(X_test_seq, padding='post')
X_test_padded = pad_sequences(X_test_padded, maxlen=500, truncating='post')

In [36]:
X_test_padded.shape

(5193, 500)

In [37]:
test

,text,label,clean_news
0,"PALO ALTO, Calif. — After years of scorning...",0,palo alto calif — years scorning political pro...
1,Russian warships ready to strike terrorists ne...,1,russian warships ready strike terrorists near ...
2,Videos #NoDAPL: Native American Leaders Vow to...,0,videos nodapl native american leaders vow stay...
3,"If at first you don’t succeed, try a different...",1,first succeed try different sport tim tebow he...
4,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,1,42 mins ago 1 views 0 comments 0 likes first t...
...,...,...,...
5195,Of all the dysfunctions that plague the world’...,0,dysfunctions plague world’s megacities none ma...
5196,WASHINGTON — Gov. John Kasich of Ohio on Tu...,1,washington — gov john kasich ohio tuesday sign...
5197,Good morning. (Want to get California Today by...,0,good morning want get california today email j...
5198,« Previous - Next » 300 US Marines To Be Deplo...,1,« previous next » 300 us marines deployed russ...


In [38]:
score = model.evaluate(X_test_padded, test['label'])

163/163 ━━━━━━━━━━━━━━━━━━━━ 24s 143ms/step - accuracy: 0.6251 - loss: 2.4175


In [47]:
model.fit(X_padded, np.array(train['label']), epochs=50)

Epoch 1/50
649/649 ━━━━━━━━━━━━━━━━━━━━ 477s 734ms/step - accuracy: 0.6556 - loss: 0.6198
Epoch 2/50
649/649 ━━━━━━━━━━━━━━━━━━━━ 478s 736ms/step - accuracy: 0.9392 - loss: 0.1673
Epoch 3/50
595/649 ━━━━━━━━━━━━━━━━━━━━ 39s 735ms/step - accuracy: 0.9903 - loss: 0.0342

KeyboardInterrupt: 

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential

In [5]:
sentences = ["I love this product", "This product is bad"]
labels = [1, 0]

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

In [12]:
word_index

{'this': 1, 'product': 2, 'i': 3, 'love': 4, 'is': 5, 'bad': 6}

In [13]:
vocab_size

7

In [7]:
X_seq = tokenizer.texts_to_sequences(sentences)
X_padded = pad_sequences(X_seq, padding='post')

In [14]:
X_padded

array([[3, 4, 1, 2],
       [1, 2, 5, 6]], dtype=int32)

In [8]:
embedding_dim = 100  # 100D GloVe vectors
embedding_index = {}

In [9]:
with open(r"C:\Projects\ml_experiments\Datasets\fake_news\glove.6B.100d.txt", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embedding_index[word] = coefs

In [10]:
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if word in embedding_index:
        embedding_matrix[i] = embedding_index[word]